I'll be using the sample data for NIFTII images

In [3]:
!git clone https://github.com/siddhanttt2506/Brain-Spy.git

Cloning into 'Brain-Spy'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64 (from 1)
Receiving objects: 100% (64/64), 192.34 MiB | 28.05 MiB/s, done.
Resolving deltas: 100% (9/9), done.
Filtering content: 100% (5/5), 347.63 MiB | 87.18 MiB/s, done.


In [2]:
import os
import numpy as np
import nibabel as nib
import pydicom
import matplotlib.pyplot as plt
import SimpleITK as sitk
from ipywidgets import interact
from pydicom.data import get_testdata_files

# 1.How to Load NIfTI and DICOM Files

This notebook is build on kaggle notebooks. I've shared the link in the Readme.md file incase you want to run the cells. For DICOM I am using data from the "OSIC Pulmonary Fibrosis Progression" contest on Kaggle.

In [6]:
nii_file = nib.load('/kaggle/working/Brain-Spy/Week 2/Sample Data/sub-62038_ses-1_acq-t1csmp2ragesag06mmUNIDEN_T1w.nii')
nii_data = nii_file.get_fdata()
nii_affine = nii_file.affine

dicom_dir = '/kaggle/input/osic-pulmonary-fibrosis-progression/train/ID00007637202177411956430'
dicom_slices = [pydicom.dcmread(os.path.join(dicom_dir, f)) for f in os.listdir(dicom_dir)]
len(dicom_slices)

30

# The internal structure and metadata

## NIFTII

In [7]:
print("Shape:", nii_data.shape)
print("Affine:\n", nii_affine)
print("Voxel Dimensions:", nii_file.header.get_zooms())
print("Data Type:", nii_data.dtype)

Shape: (256, 362, 384)
Affine:
 [[ 6.29074156e-01 -1.33072212e-02 -3.11387163e-02 -7.36054077e+01]
 [ 1.63442213e-02  6.21981978e-01  5.91654330e-02 -1.00073349e+02]
 [ 2.99664345e-02 -5.98863661e-02  6.21413589e-01 -1.19990189e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Voxel Dimensions: (0.63, 0.625, 0.625)
Data Type: float64


## DICOM meta-data

In [10]:
ds = dicom_slices[0]
print(ds)

Dataset.file_meta -------------------------------
(0002,0000) File Meta Information Group Length  UL: 200
(0002,0001) File Meta Information Version       OB: b'\x00\x01'
(0002,0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002,0003) Media Storage SOP Instance UID      UI: 2.25.46123781426224593247761081441512910006
(0002,0010) Transfer Syntax UID                 UI: Explicit VR Little Endian
(0002,0012) Implementation Class UID            UI: 1.2.276.0.7230010.3.0.3.6.1
(0002,0013) Implementation Version Name         SH: 'OSIRIX_361'
(0002,0016) Source Application Entity Title     AE: 'ANONYMOUS'
-------------------------------------------------
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0008,0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008,0018) SOP Instance UID                    UI: 2.25.46123781426224593247761081441512910006
(0008,0060) Modality                            CS: 'CT'
(0008,0070) Manufacturer 

DICOM files contain a lot of metadata. This makes it clear why they are often confidntial.  
Pixel spacing is actually pretty close in nii and dcm.

# Stacking Dicom into a 3-D volume

In [12]:
dicom_slices.sort(key=lambda s: int(s.InstanceNumber))
volume = np.stack([s.pixel_array for s in dicom_slices], axis=-1)
print(volume.shape)

(512, 512, 30)


# Visualising anatomical planes

In [13]:
def show_slices(volume):
    @interact(axial=(0, volume.shape[0]-1), 
              coronal=(0, volume.shape[1]-1), 
              sagittal=(0, volume.shape[2]-1))
    def view_slices(axial=0, coronal=0, sagittal=0):
        fig, axes = plt.subplots(1, 3, figsize=(12, 4))
        axes[0].imshow(volume[axial, :, :], cmap='gray')
        axes[0].set_title(f'Axial Slice {axial}')
        axes[1].imshow(volume[:, coronal, :], cmap='gray')
        axes[1].set_title(f'Coronal Slice {coronal}')
        axes[2].imshow(volume[:, :, sagittal], cmap='gray')
        axes[2].set_title(f'Sagittal Slice {sagittal}')
        plt.show()

print("\nInteractive Visualization of NIfTI Volume:")
show_slices(nii_data.astype(np.float32))
show_slices(volume)


Interactive Visualization of NIfTI Volume:


interactive(children=(IntSlider(value=0, description='axial', max=255), IntSlider(value=0, description='corona…

interactive(children=(IntSlider(value=0, description='axial', max=511), IntSlider(value=0, description='corona…

We have only 30 slices in dcm images. So it doesn't make much sense to plot axial and coronal projections.

# 6. Interpreting Image Orientation

### The affine matrix of NIFTII file

In [14]:
print("Affine:\n", nii_affine)

Affine:
 [[ 6.29074156e-01 -1.33072212e-02 -3.11387163e-02 -7.36054077e+01]
 [ 1.63442213e-02  6.21981978e-01  5.91654330e-02 -1.00073349e+02]
 [ 2.99664345e-02 -5.98863661e-02  6.21413589e-01 -1.19990189e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


### The orientation tags in dcm

In [21]:
print(ds.ImageOrientationPatient)

[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]


### Orientation Details

#### NIfTI Affine Matrix:

```text
[[ 6.29074156e-01 -1.33072212e-02 -3.11387163e-02 -7.36054077e+01]
 [ 1.63442213e-02  6.21981978e-01  5.91654330e-02 -1.00073349e+02]
 [ 2.99664345e-02 -5.98863661e-02  6.21413589e-01 -1.19990189e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
```

* The top-left 3×3 part of this matrix encodes how voxel axes (i, j, k) are oriented and scaled in physical space.
* The last column gives the real-world coordinates of the image origin (top-left corner of the first voxel).
* In this case, voxel sizes are approximately `0.63 mm` in each direction, with slight shearing.
* The image origin is at approximately `(-73.6, -100.1, -119.9)` in world space.

#### 📐 DICOM Orientation Tag:

```text
[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]
```

* This indicates the direction cosines of the image axes:

  * The **first three values** `[1.0, 0.0, 0.0]` represent the **row direction** in patient coordinates (left to right).
  * The **next three** `[0.0, 1.0, 0.0]` represent the **column direction** (posterior to anterior).
* This means the DICOM image is aligned with the standard patient coordinate system, with no tilt or rotation.

#### 🔁 Summary:

* The NIfTI affine gives a more detailed spatial mapping, including origin, scaling, and orientation.
* The DICOM orientation tag is simpler and more intuitive for standard axial scans but lacks origin info.
